# Assignment 2


You must submit your notebook by running `python3 -m autograder.run.submit Assignment2.ipynb` from your local repository.

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/)

Before you turn this problem in, make sure everything runs as expected. To do so, restart the kernel and run all cells (in the menubar, select Runtime→→Restart and run all).

#### Show your work!
Whenever you are asked to find the solution to a problem, be sure to also **show how you arrived** at your answer.

Make sure you fill in any place that says "YOUR CODE HERE" or "YOUR ANSWERS HERE", as well as your name below:

## Some Helper Functions

Implement an evaluation function that takes in some board position and player colour and returns a score. 

In [11]:
import chess
import random
from math import inf
from IPython.display import display, clear_output

In [12]:
# add bous points for checkmates 
def evaluation(b: chess.Board):
    """
    This function evaluates a board position and returns a score value.
    
    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the color of the active player (True -> white, False -> black)
    
    Returns:
    - an integer score for a board state
    """
    # Define weights for each factor in the linear function
    
    
    WEIGHT_MATERIAL = 1.0
    WEIGHT_MOBILITY = 0.1 
    WEIGHT_STRUCTURE = 0.5
    WEIGHT_ATTACK = 1.0
    
    # Compute the individual evaluation components
    material_score = evaluate_material(b)
    mobility_score = evaluate_mobility(b)
    attack_score = evaluate_attack_score(b)
    total_weight = (WEIGHT_MATERIAL * material_score) + (WEIGHT_MOBILITY * mobility_score) + (WEIGHT_ATTACK * attack_score)
    
    return total_weight

def evaluate_material(b:chess.Board):
    piece_values = {
        chess.PAWN: 1,
        chess.KNIGHT: 3,
        chess.BISHOP: 3,
        chess.ROOK: 5,
        chess.QUEEN: 9,
        chess.KING: 200  # King is invaluable
    }

    score = 0
    for piece, value in piece_values.items():
        score += len(list(b.pieces(piece, chess.WHITE))) * value 
        score -= len(list(b.pieces(piece, chess.BLACK))) * value 
    return score * (1 if b.turn else -1)

def evaluate_mobility(b:chess.Board):
    return len(list(b.legal_moves)) * (1 if b.turn else -1)

def evaluate_attack_score(b:chess.Board):
    CHECKMATE_BONUS = 100
    CHECK_BONUS = 5
    if b.is_check():
        return CHECK_BONUS if b.turn == chess.WHITE else -CHECK_BONUS  

    if b.is_checkmate():
        return CHECKMATE_BONUS if b.turn == chess.WHITE else -CHECKMATE_BONUS

    return 0


## Q1

Implement the minimax algorithm to choose the best chess move for a given board position, player colour, and look-ahead depth. Make use of your `evaluation()` function here to calculate the score of any individual board state.

In [13]:
def get_minimax_move(b: chess.Board, player: bool, depth: int):   
    """
    This function chooses the best move for the given board position, player, and depth.
    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    - depth: the number of moves that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """
    # limit to a depth of 6-8 to not fry computer

    def helper(local_b:chess.Board, local_player:bool, local_depth:int, alpha:float, beta:float):
        # YOUR CODE HERE
        if local_depth == 0 or local_b.outcome():
            return evaluation(local_b)
        if(local_player == chess.WHITE):
            v = float('-inf')
            for move in local_b.legal_moves:
                local_b.push(move)
                v =  max(v, helper(local_b, chess.BLACK, local_depth-1, alpha, beta))
                local_b.pop()
                alpha = max(alpha, v)
                if alpha >= beta:
                    break
            return v
        if(local_player == chess.BLACK):
            v = float('inf')
            for move in local_b.legal_moves:
                local_b.push(move)
                v = min(v, helper(local_b, chess.WHITE, local_depth-1, alpha, beta))
                local_b.pop()
                beta = min(beta, v)
                if alpha >= beta:
                    break
            return v

    alpha = float("-inf")
    beta = float("inf")
    best_move = None
    best_eval = float("-inf")
    # YOUR CODE HERE
    for move in b.legal_moves:
        b.push(move)
        eval = helper(local_b=b, local_player= not player, local_depth=depth, alpha=alpha, beta=beta)
        b.pop()
        if(player == chess.WHITE):
            if(eval > best_eval):
                best_eval, best_move = eval, move
            alpha = max(alpha, best_eval)
        if(player == chess.BLACK):
            if(eval < best_eval):
                 best_eval, best_move = eval, move
            beta = min(beta, best_eval)
        if(alpha >= beta): 
            break
    return best_move

## Q2

Implement the expectimax algorithm to choose the best chess move for a given board position, player colour, and look-ahead depth. Make use of your `evaluation()` function here to calculate the score of any individual board state.

In [14]:
def get_expectimax_move(b: chess.Board, player: bool, depth: int):   
    """
    This function chooses the best move for the given board position, player, and depth.
    
    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    - depth: the number of moves that the algorithm should look ahead.
    
    Returns:
    A single chess.Move type object.
    """        
    def helper(local_b:chess.Board, local_player:bool, local_depth:int):
        # YOUR CODE HERE
        if local_depth == 0 or local_b.outcome() or len(list(local_b.legal_moves)) == 0:
            return evaluation(local_b)
        if(local_player == chess.WHITE):
            v = 0
            for move in local_b.legal_moves:
                local_b.push(move)
                v = max(v, helper(local_b, chess.BLACK, local_depth-1))
                local_b.pop()
            return v
        if(local_player == chess.BLACK):
            legal_moves = list(local_b.legal_moves)
            move_values = []
            for move in local_b.legal_moves:
                local_b.push(move)
                move_value = helper(local_b, chess.WHITE, local_depth-1)
                local_b.pop()
                move_values.append(move_value)
            expected_value = sum(move_values) / len(legal_moves)
            return expected_value

    best_move = None
    best_eval = float("-inf")
    # YOUR CODE HERE
    for move in b.legal_moves:
        b.push(move)
        eval = helper(local_b=b, local_player=player, local_depth=depth-1)
        b.pop()
        if(player == chess.WHITE):
            if(eval > best_eval):
                best_eval, best_move = eval, move

    return best_move
        

## Q3

Plays against the strongest grading agent, so copy your best agent here.

In [15]:
def get_best_move(b: chess.Board, player: bool, depth: int):   
    """
    This function chooses the best move for the given board position, player, and depth.
    
    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    - depth: the number of moves that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """    
    
    # YOUR CODE HERE
    return get_minimax_move(b, player, depth)

## Local Testing
For your convenience, here are some helper functions to run games and allow you to test your agents.

In [16]:
def get_random_move(b:chess.Board, *_):
    rand = random.Random(0)
    return rand.choice(list(b.legal_moves))

In [17]:
MAX_DEPTH = 3
SEED = random.Random(0)  # Feel free to set/reset the seed for testing purposes!

def get_random_move(b:chess.Board, *_):
    return SEED.choice(list(b.legal_moves))

def test_game(p1, p2):
    """
    A function that plays a chess game with visuals
    
    Parameters:
    - p1: the first chess agent, playing as the white player
    - p2: the second chess agent, playing as the black player
    
    Returns:
    Nothing, just plays the game one move at a time.
    Press enter in the popup box to play the next move.
    Type 'q' into the popup box and press enter to stop the game (you can also interrupt the kernel if this fails.)
    """
    board = chess.Board()
    current_player = p1

    while not board.is_checkmate()  and not board.is_stalemate() and not board.can_claim_draw():
        clear_output(True)
        
        move = current_player(board, board.turn, MAX_DEPTH)
        board.push(move)
        display(board)

        current_player = p1 if board.turn else p2

        if input() == "q":
            break
    

In [18]:
# Random Vs. Random
random_agent = get_random_move
# test_game(random_agent, random_agent)

In [19]:
# Minimax Vs. Random
random_agent = get_random_move
minimax_agent = get_minimax_move
# test_game(minimax_agent, random_agent)

In [20]:
# Expectimax Vs. Random
random_agent = get_random_move
expectimax_agent = get_expectimax_move
# test_game(expectimax_agent, random_agent)

In [21]:
# Run several games at once without graphics
def grade_game(p1, p2):
    """
    Grades a single game at a time.
    (Note: differs from actual grading script, which has some ways to resolve draws.)

    Return legend:
    0: error during game
    1: p1 wins through checkmate
    2: p2 wins through checkmate
    3: draw
    """
    board = chess.Board()
    current_player = p1

    while not board.is_checkmate() and not board.is_stalemate() and not board.can_claim_draw():
        move = current_player(board,board.turn, MAX_DEPTH)
        try:
            board.push(move)
        except:            
            print(f"Error while grading game, move = {move}, current_player = {current_player}")
            print(board)
            return -1

        current_player = p1 if board.turn else p2
    
    outcome = board.outcome()
    if outcome is not None:
        if outcome.winner:
            return 1
        else:
            return 2
    
    return 3

In [22]:
# Play many games in a row without visuals (faster)
random_agent = get_random_move
best_agent = get_best_move
num_games = 5

print(f"Playing {num_games} games...")
for i in range(1, num_games+1):
    result = grade_game(best_agent, random_agent)

    if result == 0:
        print(f"Game {i}: error pushing a move during gameplay")
        
    elif result == 1:
        print(f"Game {i}: p1 wins")
        
    elif result == 2:
        print(f"Game {i}: p1 loses")
    
    elif result == 3:
        print(f"Game {i}: draw")

Playing 5 games...
Game 1: draw
Game 2: draw
Game 3: draw
Game 4: draw
Game 5: draw
